In [ ]:
!pip install pandas selenium webdriver_manager tqdm

     |████████████████████████████████| 904 kB 8.5 MB/s 


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
import os
import pandas as pd
import selenium
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options  # for suppressing the browser
from csv import writer
from tqdm import tqdm
import io
import os

load_button_loop = 20
keywords_export = "google_trend_keywords.csv"
limit = 1000

country_iso2 = "https://raw.githubusercontent.com/sandeepyadav1478/keywords-typewriter/main/assets/country.csv"

keywords = []
counter = 0
counter_country = 0

if not os.path.isfile(keywords_export):
  f = open(keywords_export, "w")
  f.write("title")
  f.close()
  keywords.append('title')

print("\nChecking driver in catch or Downloading it ...")
option = webdriver.ChromeOptions()
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=option)

def lets_fire(q):
    search_url="https://trends.google.com/trends/trendingsearches/daily?geo={q}" 
    print("\nSending request for country iso2 code :",q)
    driver.get(search_url.format(q=q))
    for i in tqdm(range(0,load_button_loop), position=0, leave=True):
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        try:
            driver.find_element_by_xpath('//*[@class="feed-load-more-button"]').click()
        except NoSuchElementException as er:
            print("\nError : load more button not available \n",er)
            break
    Results = driver.find_elements_by_xpath('//*[@class="title"]/span/a')
    with open(keywords_export, 'a') as f_object:
      writer_object = writer(f_object)
      for a in Results:
          global counter
          if a.text not in keywords:
            counter = counter + 1
            keywords.append(a.text)
            writer_object.writerow([a.text])
      print("\nTotal Found now ",counter)
      f_object.close()

keywords.append(list(pd.read_csv(keywords_export)))
country_iso2_data = pd.read_csv(country_iso2)
for i in country_iso2_data["iso2"]:
  if counter > limit:
    print("Set limit reached.")
    break
  print((counter_country/48)*100,"% completed.")
  lets_fire(i)
  limit += 1
  counter_country += 1

print("quiting driver ...")
driver.quit()

print("\n Total numbers of keywords found on google trend : ",counter)


# print("\n Saving data in ...",keywords_export)
# keywords_df = pd.DataFrame(keywords)
# keywords_df.columns = ['title']
# keywords_df.to_csv(keywords_export,index=False)




Checking driver in catch or Downloading it ...
0.0 % completed.

Sending request for country iso2 code : AR


100%|██████████| 20/20 [00:44<00:00,  2.24s/it]



Total Found now  351
2.083333333333333 % completed.

Sending request for country iso2 code : AU


100%|██████████| 20/20 [00:44<00:00,  2.25s/it]



Total Found now  688
4.166666666666666 % completed.

Sending request for country iso2 code : AT


100%|██████████| 20/20 [00:44<00:00,  2.25s/it]



Total Found now  1019
quiting driver ...

 Total numbers of keywords found on google trend :  1019
